_READ ME_

This script is designed to help Forum Analysts sort postal codes into pre-defined geographic areas (municipalities, wards, etc) quickly and accurately. This specific script is designed to sort Oakville postal codes into the city's wards, but can be easily tweaked for any other area in Canada and any other geographic partition. To do so, you will need: 
- a database of postal codes and associated coordinates (in this script, I use the postal code database from Service Objects - available for free! - https://www.serviceobjects.com/blog/free-zip-code-and-postal-code-database-with-geocoordinates/). They update this database regularly so you may want to replace the file. 
- geoJSON files of the geographic areas you wish to sort your postal codes into (this script was written for geoJSON files but could easily be adapted for any other geospatial data file). Depending on the area you are trying to sort your postal codes into, you may need to create your own geoJSON files using https://geojson.io/#map=2/20.0/0.0, or you may be able to find the geospatial data you need in a public database. 

The following are information you will need to feed to the program as it runs: 
- the path of the CSV file containing the postal codes you wish to sort
- the name of the city or cities that 

In [1]:
#load packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

C:\Users\macla\anaconda3.2\envs\geopandas_env\lib\site-packages\pyproj\__init__.py:91: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


In [2]:
#Store path of Service Objects database with postal codes and coordinates in a variable
database_path = 'CanadianPostalCodes202201.csv' #change this to the path of your Service Objects database
all_postal_codes_df = pd.read_csv(database_path)

#Generate list of cities in the Service Objects database that have postal code information
city_options = all_postal_codes_df['CITY'].unique()
city_options_string = str(city_options)
city_options

array(['ETOBICOKE', 'BARRHEAD', 'LAVAL', ..., 'SHINING TREE', 'DEEP BAY',
       'TOWNSEND'], dtype=object)

In [3]:
#This function creates a geodataframe with the postal code and geometry information for the city we want to study
def make_city_gdf():
    make_city_gdf.city = input('Enter the name of the city you want to create a geodataframe for:')
    if make_city_gdf.city.upper() in city_options: #Check that data for this city is available by searching the list of all cities in the database.
        make_city_gdf.city_of_interest_df = all_postal_codes_df.loc[all_postal_codes_df['CITY']==make_city_gdf.city.upper()] #Create a new dataframe with the data for the city of interest
        make_city_gdf.city_of_interest_df = make_city_gdf.city_of_interest_df.drop(columns=['PROVINCE_ABBR','TIME_ZONE']) #Drop unnecessary columns
        make_city_gdf.city_of_interest_df['POSTAL_CODE'] = make_city_gdf.city_of_interest_df['POSTAL_CODE'].str.replace(" ","") #Delete the space between characters of the postal code
        make_city_gdf.city_gdf = gpd.GeoDataFrame(make_city_gdf.city_of_interest_df, geometry=gpd.points_from_xy(make_city_gdf.city_of_interest_df['LONGITUDE'], make_city_gdf.city_of_interest_df['LATITUDE'])) #Convert the dataframe to a geodataframe
        return make_city_gdf.city_gdf #Return a geodataframe corresponding to the city of interest
    else: 
        print('Data for this city is not available in the Service Objects database.') #Error message if the city of interest is not in the database
        
oakville = make_city_gdf()

Enter the name of the city you want to create a geodataframe for: oakville


In [4]:
oakville

,POSTAL_CODE,CITY,LATITUDE,LONGITUDE,geometry
219,L6J6T3,OAKVILLE,43.505564,-79.661156,POINT (-79.66116 43.50556)
411,L6J1A7,OAKVILLE,43.453407,-79.655157,POINT (-79.65516 43.45341)
438,L6J3E8,OAKVILLE,43.446245,-79.664485,POINT (-79.66448 43.44624)
537,L6J5K7,OAKVILLE,43.481283,-79.648956,POINT (-79.64896 43.48128)
930,L6M3Z5,OAKVILLE,43.449534,-79.710710,POINT (-79.71071 43.44953)
...,...,...,...,...,...
893903,L6H6L1,OAKVILLE,43.460427,-79.738772,POINT (-79.73877 43.46043)
893974,L6H1E7,OAKVILLE,43.455144,-79.705165,POINT (-79.70517 43.45514)
894337,L6K3Y2,OAKVILLE,43.437070,-79.680799,POINT (-79.68080 43.43707)
894407,L6M4G8,OAKVILLE,43.448441,-79.794108,POINT (-79.79411 43.44844)


In [5]:
#Create df with the postal codes we are interested in mapping to wards. user is asked to input the path of the file containing the postal codes they wish to study

#This function creates a dataframe of the postal codes in our CATI/CAWI file that need to be sorted into wards
path = r"C:\Users\macla\Desktop\ward_coding\CATI_file.csv" #Change this path to your file
data_df = pd.read_csv(path) #Store CAWI/CATI data in a df
fsa_column = 'FSAR1M1' #Change this to the name of the FSA column in your data file
postal_code_column = 'D8R1M1' #Change this to the name of the postal code column in your data file
id_column = 'resRespondent' #Change this to the name of the respondent ID column in your data file

codes_to_sort_df = data_df[[id_column,fsa_column,postal_code_column]] #Create a dataframe with just the FSAs and postal codes to sort
codes_to_sort_df = codes_to_sort_df.rename(columns={id_column: 'Respondent ID', fsa_column: 'FSA', postal_code_column: 'Postal code'}) #Rename the columns for clarity
codes_with_geom_df = pd.merge(codes_to_sort_df, oakville, how='left', left_on='Postal code',right_on='POSTAL_CODE') #Merge the geodataframe of postal
#codes to sort with the latitude and longitude information for each postal code
codes_with_geom_gdf = gpd.GeoDataFrame(codes_with_geom_df, geometry='geometry')

In [6]:
#Create a geodf with the polygons for all 7 wards in Oakville
all_wards_gdf=gpd.read_file(r"C:\Users\macla\Desktop\ward_coding\allwards.geojson") #Change path to geojson datafile with all the wards or all the geospatial data you wish to use

#Add a ward column to the polygon dataframe
wards_df = pd.DataFrame(all_wards_gdf)
wards = [1, 2, 3, 4, 5, 6, 7]
wards_df['Ward']=wards
wards_gdf = gpd.GeoDataFrame(wards_df, geometry=wards_df['geometry'])
wards_gdf

,geometry,Ward
0,"POLYGON ((-79.72328 43.37093, -79.71641 43.374...",1
1,"POLYGON ((-79.72538 43.42391, -79.72044 43.421...",2
2,"POLYGON ((-79.64787 43.52428, -79.67266 43.501...",3
3,"POLYGON ((-79.70976 43.43784, -79.71145 43.438...",4
4,"POLYGON ((-79.75147 43.45937, -79.74928 43.458...",5
5,"POLYGON ((-79.68364 43.46133, -79.67594 43.468...",6
6,"POLYGON ((-79.71323 43.49365, -79.74134 43.514...",7


In [7]:
#Add an FSA column to the gdf of city information
make_city_gdf.city_gdf['FSA'] = make_city_gdf.city_gdf['POSTAL_CODE'].str[:3]
make_city_gdf.city_gdf = gpd.sjoin(make_city_gdf.city_gdf, wards_gdf, how='left')

In [8]:
#Make a list of all the unique FSAs in the gdf of city information
fsa_list = make_city_gdf.city_gdf['FSA'].unique()

#Create a new list, and if the FSA starts with L (FSAs in Oakville), append that FSA to the new list
new_fsa_list=[]
for item in fsa_list:
    if item[:1] == 'L':
        new_fsa_list.append(item)
new_fsa_list

['L6J', 'L6M', 'L6L', 'L6H', 'L6K']

In [9]:
#Join the ward df with the df of all our postal codes to be sorted
final_df = gpd.sjoin(codes_with_geom_gdf, wards_gdf, how='left')
pd.set_option('display.max_rows', 1000)

In [10]:
probs ={} #Create a dictionary where we store the probability of a each FSA 

probs_df = pd.DataFrame(columns=['FSA']) 

#This for loop iterates through every FSA in Oakville and returns dictionaries with the probability that each 
for item in new_fsa_list: #For every FSA in oakville
    fsa_df = make_city_gdf.city_gdf.loc[make_city_gdf.city_gdf['FSA']==item] #Make a temporary data frame with the postal codes of that FSA
    total = fsa_df['POSTAL_CODE'].unique() 
    length_total = len(total) #Get total number of postal codes in that FSA
    wards = range(1,8)
    probs.update({'FSA':item}) #Update the probs dictionary with the FSA number
    for ward in wards: #For every ward in the list of wards
        if ward in fsa_df.Ward.values: #If the ward is in the dataframe of the FSA
            prob = fsa_df.Ward.value_counts()[ward] 
            probs[ward] = prob/length_total #The probability of the FSA being in that ward is equal to the number of 
        else: 
            prob = 0 #Otherwise, the probability of the FSA being in that ward is 0
            probs[ward] = 0
    temp_df = pd.DataFrame([probs]) #Transform the dictionary into a dataframe
    probs_df = pd.concat([probs_df, temp_df], ignore_index=True) #Concatenate the 2 dataframes 
    
probs_df #Return the dataframe of probabilities of each FSA being in which ward. 

,FSA,1,2,3,4,5,6,7
0,L6J,0.000831,0.000000,0.995017,0.000831,0.001661,0.000000,0.000831
1,L6M,0.053917,0.204476,0.007121,0.663276,0.015259,0.004069,0.044761
2,L6L,0.605860,0.384688,0.002836,0.002836,0.000945,0.000000,0.000000
3,L6H,0.005835,0.000729,0.001459,0.006565,0.431072,0.493800,0.068563
4,L6K,0.000000,0.986038,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
import numpy as np
final_df=final_df.rename(columns={'Postal code':'Postal_code'})
#If the postal code column is PNA, the FSA is the FSA column
#Else, the FSA is the first three characters of the postal code
final_df['Actual_FSA'] = pd.np.where(final_df.Postal_code.str.contains('Prefer not to say'), final_df['FSA'],final_df['Postal_code'].str[:3])

C:\Users\macla\AppData\Local\Temp\ipykernel_8588\602724865.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  final_df['Actual_FSA'] = pd.np.where(final_df.Postal_code.str.contains('Prefer not to say'), final_df['FSA'],final_df['Postal_code'].str[:3])


In [12]:
import random

#Create lists of probability distribution weights for each FSA
copy_probs_df = probs_df.drop(columns=['FSA']) #Make a copy of probs_df
probs_list = copy_probs_df.values.tolist() #Convert all rows of the dataframe to lists stored in the probs_list variable

wards = [1, 2, 3, 4, 5, 6, 7] #Create list of wards

In [13]:
#This function allows us to sort all PNA postal codes into Wards based on the probability that the FSA is in that ward. 
def fill_PNA(postal_code, fsa, ward):
    pna='Prefer not to say'
    if postal_code==pna and fsa=='L6K':
        ward = 2
    elif postal_code==pna and fsa=='L6J':
        ward = 3
    elif postal_code==pna and fsa=='L6M': 
        ward_list = random.choices(wards, probs_list[1]) #Randomly assign to a ward using the probability of the FSA being in each of the wards
        ward = ward_list[0] #convert list to integer
    elif postal_code==pna and fsa=='L6L':
        ward_list = random.choices(wards, probs_list[2])
        ward = ward_list[0]
    elif postal_code==pna and fsa=='L6H':
        ward_list = random.choices(wards, probs_list[3])
        ward = ward_list[0]
    else:
        ward = ward
    return ward

#We use the apply method and lambda function. This takes as input our function with our pandas cols as parameters. Use axis=1 to apply the function row-wise
final_df['Ward_complete']=final_df.apply(lambda x: fill_PNA(x['Postal_code'], x['Actual_FSA'], x['Ward']), axis=1)

In [14]:
unfound_df = final_df.loc[(final_df['Ward_complete'].isna())]#Create a df of values that couldn't be found 
unfound_list = unfound_df['Postal_code'].tolist()

In [15]:
final_df['Ward_complete'] = final_df['Ward_complete'].fillna(0) #Fill NaN values with 0

#There are several groups of unsorted postal codes at this moment: 
#1. Postal codes that were provided but that did not match a postal code in the Service Objects database
#2. Postal codes with an FSA that did not match any FSA in Oakville
#This function allows us to sort all postal codes that have not yet been sorted into wards based on the FSA
def fill_NaN(ward, actual_fsa, approx_fsa):
    if actual_fsa in new_fsa_list: #If the actual FSA (drawn from the postal code provided is an FSA in Oakville), randomly assign to a ward based on probability
        NaN=0
        if ward==NaN and actual_fsa=='L6K':
            ward = 2
        elif ward==NaN and actual_fsa=='L6J':
            ward = 3
        elif ward==NaN and actual_fsa=='L6M': 
            ward_list = random.choices(wards, probs_list[1])
            ward = ward_list[0] 
        elif ward==NaN and actual_fsa=='L6L':
            ward_list = random.choices(wards, probs_list[2])
            ward = ward_list[0]
        elif ward==NaN and actual_fsa=='L6H':
            ward_list = random.choices(wards, probs_list[3])
            ward = ward_list[0]
        else:
            ward = ward
        return ward
    else: #If the actual FSA is not an FSA in Oakville, we use the approx FSA provided by the original dataset to sort
        NaN=0
        if ward==NaN and approx_fsa=='L6K':
            ward = 2
        elif ward==NaN and approx_fsa=='L6J':
            ward = 3
        elif ward==NaN and approx_fsa=='L6M': 
            ward_list = random.choices(wards, probs_list[1])
            ward = ward_list[0] 
        elif ward==NaN and approx_fsa=='L6L':
            ward_list = random.choices(wards, probs_list[2])
            ward = ward_list[0]
        elif ward==NaN and approx_fsa=='L6H':
            ward_list = random.choices(wards, probs_list[3])
            ward = ward_list[0]
        else:
            ward = ward
        return ward

#We use the apply method and lambda function. This takes as input our function with our pandas cols as parameters. Use axis=1 to apply the function row-wise
final_df['Ward_complete']=final_df.apply(lambda x: fill_NaN(x['Ward_complete'], x['Actual_FSA'], x['FSA']), axis=1)
final_df.drop(columns=['Respondent ID'], inplace=True) #Omit respondent ID for privacy reasons
#Would need to leave this column to sort 

,FSA,Postal_code,POSTAL_CODE,CITY,LATITUDE,LONGITUDE,geometry,index_right,Ward,Actual_FSA,Ward_complete
0,L6L,L6L2T1,L6L2T1,OAKVILLE,43.424057,-79.702332,POINT (-79.70233 43.42406),1.0,2.0,L6L,2.0
1,L6L,L6K3R6,L6K3R6,OAKVILLE,43.449257,-79.685661,POINT (-79.68566 43.44926),1.0,2.0,L6K,2.0
2,L6K,L6M5C6,L6M5C6,OAKVILLE,43.445523,-79.718448,POINT (-79.71845 43.44552),1.0,2.0,L6M,2.0
3,L6M,L6M0C6,L6M0C6,OAKVILLE,43.437553,-79.753286,POINT (-79.75329 43.43755),3.0,4.0,L6M,4.0
4,L6J,L6J6X5,L6J6X5,OAKVILLE,43.501389,-79.665075,POINT (-79.66508 43.50139),2.0,3.0,L6J,3.0
5,L6M,L6M4A4,L6M4A4,OAKVILLE,43.449075,-79.748323,POINT (-79.74832 43.44908),3.0,4.0,L6M,4.0
6,L6J,L6K1R3,L6K1R3,OAKVILLE,43.433997,-79.691265,POINT (-79.69127 43.43400),1.0,2.0,L6K,2.0
7,L6J,L6H0G2,L6H0G2,OAKVILLE,43.497608,-79.705367,POINT (-79.70537 43.49761),5.0,6.0,L6H,6.0
8,L6H,L6H5C8,L6H5C8,OAKVILLE,43.465269,-79.698030,POINT (-79.69803 43.46527),4.0,5.0,L6H,5.0
9,L6L,L6L6L9,L6L6L9,OAKVILLE,43.391727,-79.714344,POINT (-79.71434 43.39173),0.0,1.0,L6L,1.0


In [17]:
#Add wards back into original dataframe
condensed_final_df = final_df[['Postal_code', 'Ward_complete']] #create dataframe with just respondent IDs and wards

#merge the condensed df with the original data
results_df = pd.merge(data_df,
                      condensed_final_df,
                      left_on='Postal_code',
                      right_on='postal_code_column,
                      how='left')

#Create copy of original CSV CATI/CAWI data file
import shutil 
copy = r"C:\Users\macla\Desktop\ward_coding\coded_wards.csv" #Change this to the destination you want to save the file and the name you want to give the file
shutil.copyfile(path, copy) #path was our original csv file

#Export final df to copied file
results_df.to_csv(r"C:\Users\macla\Desktop\ward_coding\coded_wards.csv", index=False) #Change this to your output file

KeyError: 'postal_code_column'